In [1]:
import importlib
import GUI_notebook
importlib.reload(GUI_notebook)
vb = GUI_notebook.init(In)
vb

In [12]:
"""
---Test Text---

# Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

# Component2

long_variable = a + b #[long_unit]
function_test = np.log(x + np.sin(y)) #[rad]


"""

'\n---Test Text---\n\n# Component1\n\nx = 2*y + 3*z #[m]\na = b + c*1.2e-3 #[kg]\n\n# Component2\n\nlong_variable = a + b #[long_unit]\nfunction_test = np.log(x + np.sin(y)) #[rad]\n\n\n'

In [ ]:
"""
---Test Text---
## Group1

# Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]

## Group2

# Component3

x = 2*y + 3*z
a = b + c

# Component4

long_variable = a + b
function_test = np.log(x + np.sin(y))

"""

In [2]:
# Exclude
x = 0
z = 1

In [3]:
# Component1
y = x + 1
y = y**2 + z

In [4]:
y

2

In [5]:
## Group1

# Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

NameError: name 'b' is not defined

In [6]:
# Exclude
a = 0
b = 1

In [7]:
# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]

NameError: name 'np' is not defined